In [ ]:
#Import Statement
import PyPDF2
import openpyxl
from openpyxl import load_workbook
import shutil
import fitz
from PyPDF2 import PdfReader
import os
import pandas as pds

In [ ]:
#Function Modules
##############################################################################
def find_pages_with_keyword(path,keyword2):
    try:
        doc=fitz.open(path)
        pages_with_keyword=[]
        for page_no in range(doc.page_count):
            page=doc[page_no]
            text=page.get_text()

            if keyword2 in text:
                pages_with_keyword.append(page_no)
        return pages_with_keyword if pages_with_keyword else None
    except Exception as e:
        print(f"An error ocurred {e}")
        return None

#################################################################################

def count_keyword_occurrences_for_pages(path,keyword,page_indices):
    try:
        doc=fitz.open(path)
        keyword_counts={}

        for page_no in page_indices:
            if isinstance(page_no,int) and 0 <= page_no <doc.page_count:
                page=doc[page_no]
                text=page.get_text()
                keyword_count=text.count(keyword)
                #print(keyword_count)
                keyword_counts[page_no]=keyword_count

            else:
                print("page no is out of range")
        return keyword_counts if keyword_counts else {}
    except Exception as e:
        print(f"An error occurred:{e}")
        return None

#################################################################################

def perform_task_on_pages(path,page_index):
    try:
        doc=fitz.open(path)
        for page_num in page_index:
            if 0<=page_num<doc.page_count:
                keyword_tocount="5.ITEM DESCRIPTION"
                page_index_to_process=result2
                res=count_keyword_occurrences_for_pages(path,keyword_tocount,page_index)

                if res is not None:
                    return res
                else:
                    print("failed")
            else:
                print("out of range")
    except Exception as e:
        print(f"an error occurred{e}")

####################################################################################

def find_pdf_page_indices(path,keyword):

    try:
        pdf_document=fitz.open(path)
        indices=[pageno for pageno in range(pdf_document.page_count) if keyword in pdf_document[pageno].get_text("text")]
        if indices:
                print(f"keyword found in {indices}")
                return indices

        else:
                print("keyword not found")
    except Exception as e:
        print(f"An error occured {e}")

###################################################################################

def check_and_operate(path,sheet_name,column_name,keyword):
    try:
        workbook=openpyxl.load_workbook(path)
        sheet=workbook[sheet_name]
        header_row=sheet[1]
        column_index=[i for i, header in enumerate(header_row,1) if header == column_name]

        for cell in sheet.iter_rows(min_row=2,min_col=column_index,max_col=column_index,values_only=True):
            if keyword == cell[0]:
                Test=5
                return Test
            else:
                Test=10
                return Test

                #print("kw already exists")
    except Exception as e:
        print(f"Exception {e}")




In [ ]:
#Scrapping data from the two different templates and appending data to the excel file
try:
            d2df=pds.DataFrame()
            files=os.fsencode(r"Source_Path/")
            if len(os.listdir(r'Source_Path/'))>0:
                for file in os.listdir(files):
                    filename=os.fsdecode(file)
                    if filename.endswith(".pdf") or filename.endswith(".PDF"):
                        path=r'Source_Path/'+filename
                        spreadsheet_path="BOE-Raw_Data.xlsx"
                        print(path)
                        keyword_to_find="PART - III - DUTIES"
                        result2=find_pages_with_keyword(path,keyword_to_find)
                        keyword="BILL OF ENTRY FOR HOME CONSUMPTION"
                        result_indices=find_pdf_page_indices(path,keyword)
                        if result2 == None and result_indices != None:
                            print("Before boe")

                            reader=PdfReader(path)
                            pages=reader.pages[0]
                            text=pages.extract_text().split("\n")
                            partial_keyword="BE No/Dt./cc/"
                            index_found=next((index for index, item in enumerate(text) if partial_keyword in item),None)
                            BOE1=str(text[index_found])
                            BOE1=BOE1.split("/")
                            BOE1=BOE1[3]
                            BOE1=str(BOE1)
                            BOE1=BOE1.split(":")
                            BOE1=BOE1[1]

                            keyword_to_check_redundency=int(BOE1)
                            print("keyword",keyword_to_check_redundency)
                            df=pds.read_excel(r"BOE-Raw_Data.xlsx")

                            is_present=keyword_to_check_redundency in df.iloc[:,0].values
                            if is_present:
                                print("Redundancy")

                            else:
                                keyword="BILL OF ENTRY FOR HOME CONSUMPTION"
                                result_indices=find_pdf_page_indices(path,keyword)
                                reader=PdfReader(path)
                                pages=reader.pages[0]
                                text=pages.extract_text().split("\n")

                                partial_keyword="BE No/Dt./cc/"
                                index_found=next((index for index, item in enumerate(text) if partial_keyword in item),None)
                                BOE=str(text[index_found])
                                BOE=BOE.split("/")
                                BOE=BOE[3]
                                BOE=str(BOE)
                                BOE=BOE.split(":")
                                BOE=int(BOE[1])

                                partial_keyword="BE No/Dt./cc/"
                                index_found=next((index for index, item in enumerate(text) if partial_keyword in item),None)
                                Date=str(text[index_found])
                                Date=Date.split(":")
                                Date=Date[1][-18:-8]

                                partial_keyword="Cntry Of Consgn"
                                index_found=next((index for index, item in enumerate(text) if partial_keyword in item),None)
                                index_found=index_found+1
                                HAWB=str(text[index_found])
                                HAWB=[word for word in HAWB.split() if word !=' ']
                                HAWB=HAWB[-1]

                                partial_keyword="Gross Wt."
                                index_found=next((index for index, item in enumerate(text) if partial_keyword in item),None)
                                Gross_wt=str(text[index_found])
                                Gross_wt=[word for word in Gross_wt.split() if word !=' ']
                                Gross_wt=Gross_wt[-2]

                                partial_keyword="No. Of Pkgs."
                                index_found=next((index for index, item in enumerate(text) if partial_keyword in item),None)
                                PKG=str(text[index_found])
                                PKG=[word for word in PKG.split() if word !=' ']
                                PKG=PKG[4]

                                partial_keyword="Gross Wt."
                                index_found=next((index for index, item in enumerate(text) if partial_keyword in item),None)
                                index_found=index_found+4
                                Invno=str(text[index_found]).split(" ")
                                Invno=Invno[5]

                                dic={'BOE':BOE,"BOEDate":Date,"HAWB":HAWB,"Invoice_no":Invno,"PKG":PKG,"Gross_Wt":Gross_wt}

                                for y in result_indices:
                                    if y==0:
                                        reader=PdfReader(path)
                                        pages=reader.pages[y]
                                        text=pages.extract_text().split("\n")

                                        partial_keyword="CETH"
                                        index_found=next((index for index, item in enumerate(text) if partial_keyword in item),None)
                                        index_found=index_found+2
                                        PN1=str(text[index_found])
                                        PN1=PN1.split(' ')
                                        PN1=PN1[7]

                                        partial_keyword="CETH"
                                        index_found=next((index for index, item in enumerate(text) if partial_keyword in item),None)
                                        index_found=index_found+4
                                        QTY1=str(text[index_found])
                                        QTY1=QTY1.split(' ')
                                        QTY1=QTY1[0]


                                        dic1={"PN":PN1,"QTY":QTY1}
                                        dic.update(dic1)
                                        d1df1=pds.DataFrame(dic,index=[0])
                                        d2df=d2df.append(d1df1,ignore_index=True)
                                        print(d2df)

                                    else:
                                        reader=PdfReader(path)
                                        pages1=reader.pages[y]
                                        text1=pages1.extract_text().split("\n")
                                        text1

                                        partial_keyword="B/F"
                                        index_found=next((index for index, item in enumerate(text1) if partial_keyword in item),None)
                                        if(index_found==None):
                                            partial_keyword="CETH"
                                            index_found=next((index for index, item in enumerate(text1) if partial_keyword in item),None)
                                            print(index_found)
                                            index_found=index_found+2
                                            PN1=str(text1[index_found])
                                            PN1=PN1.split(' ')
                                            PN1=PN1[7]
                                            index_found=index_found+2
                                            QTY1=str(text1[index_found])
                                            QTY1=QTY1.split(' ')
                                            QTY1=QTY1[0]


                                            dic2={"PN":PN1,"QTY":QTY1}
                                            dic.update(dic2)
                                            d1df3=pds.DataFrame(dic,index=[0])
                                            d2df=d2df.append(d1df3,ignore_index=True)
                                        else:
                                            partial_keyword="B/F"
                                            index_found=next((index for index, item in enumerate(text1) if partial_keyword in item),None)
                                            print(index_found)
                                            index_found=index_found+2
                                            PN1=str(text1[index_found])
                                            PN1=PN1.split(' ')
                                            PN1=PN1[7]
                                            index_found=index_found+2
                                            QTY1=str(text1[index_found])
                                            QTY1=QTY1.split(' ')
                                            QTY1=QTY1[0]

                                            dic2={"PN":PN1,"QTY":QTY1}
                                            dic.update(dic2)
                                            d1df4=pds.DataFrame(dic,index=[0])
                                            d2df=d2df.append(d1df4,ignore_index=True)


                        elif result2 !=None:
                            print("After boe")
                            keyword_to_find="PART - III - DUTIES"
                            reader=PdfReader(path)
                            pages=reader.pages[0]
                            text=pages.extract_text().split("\n")


                            BOE2=text[0]
                            BOE2=BOE2.split(' ')
                            BOE2=BOE2[0]


                            keyword_to_check_redundency=int(BOE2)
                            print("keyword",keyword_to_check_redundency)
                            df=pds.read_excel("BOE-Raw_Data.xlsx")

                            is_present=keyword_to_check_redundency in df.iloc[:,0].values
                            if is_present:
                                print("redundancy")

                            else:
                                result2=find_pages_with_keyword(path,keyword_to_find)
                                if result2 is not None:
                                    print(f"Keyword found on pages:{result2}")
                                else:
                                    print(f"keyword not found")

                                result3=perform_task_on_pages(path,result2)

                                BOE=text[0]
                                BOE=BOE.split(' ')
                                BOE=int(BOE[0])
                                BOE

                                BOEDate=text[0]
                                BOEDate=BOEDate.split(' ')
                                BOEDate=BOEDate[1]
                                BOEDate

                                PKG=text[3]
                                PKG=PKG.split(' ')
                                PKG=PKG[0]
                                PKG

                                GrossWt=text[3]
                                GrossWt=GrossWt.split(' ')
                                GrossWt=GrossWt[1][:-4]
                                GrossWt

                                hawb=""

                                pages=reader.pages[1]
                                text=pages.extract_text().split("\n")

                                partial_keyword="PART - II - INVOICE & VALUATION DETAILS "
                                index_found=next((index for index, item in enumerate(text) if partial_keyword in item),None)
                                index_found=index_found+1
                                Inv_no=str(text[index_found]).split(" ")
                                l=len(Inv_no)
                                if l>1:
                                    Inv_no=Inv_no[1]
                                    print(Inv_no)
                                elif l==1:
                                    Inv_no=Inv_no[0][1:]
                                    print(Inv_no)

                                dic={'BOE':BOE,"BOEDate":BOEDate,"HAWB":hawb,"Invoice_no":Inv_no,"PKG":PKG,"Gross_Wt":GrossWt}

                                for key,value in result3.items():
                                    pages1=reader.pages[key]
                                    text1=pages1.extract_text().split("\n")
                                    if value==1:
                                        partial_keyword="ASSESSED    COPY"
                                        index_found=next((index for index, item in enumerate(text1) if partial_keyword in item),None)
                                        PN1=str(text1[index_found])
                                        PN1=PN1.split(' ')
                                        PN1=PN1[6]
                                        PN1=PN1[4:]

                                        partial_keyword="NOEXCISE"
                                        index_found=next((index for index, item in enumerate(text1) if partial_keyword in item),None)
                                        QTY1=str(text1[index_found+1])
                                        QTY1=QTY1.split(' ')
                                        QTY1=QTY1[0]

                                        dic1={'PN':PN1,'QTY':QTY1}
                                        dic.update(dic1)
                                        d1df1=pds.DataFrame(dic,index=[0])
                                        d2df=d2df.append(d1df1,ignore_index=True)
                                    elif value==2:
                                        partial_keyword="ASSESSED    COPY"
                                        index_found=next((index for index, item in enumerate(text1) if partial_keyword in item),None)
                                        PN1=str(text1[index_found])
                                        PN1=PN1.split(' ')
                                        PN1=PN1[6]
                                        PN1=PN1[4:]

                                        PN2=text1[index_found+1]
                                        PN2=PN2.split(' ')
                                        PN2
                                        PN2=PN2[0]

                                        partial_keyword="NOEXCISE"
                                        index_found=next((index for index, item in enumerate(text1) if partial_keyword in item),None)
                                        QTY1=str(text1[index_found+1])
                                        QTY1=QTY1.split(' ')
                                        QTY1=QTY1[0][8:]

                                        partial_keyword="NOEXCISE"
                                        index_found=next((index for index, item in enumerate(text1) if partial_keyword in item),None)
                                        QTY2=str(text1[index_found+2])
                                        QTY2=QTY2.split(' ')
                                        QTY2=QTY2[0][:-3]

                                        dic1={'PN':PN1,'QTY':QTY1}
                                        dic2={'PN':PN2,'QTY':QTY2}
                                        dic.update(dic1)
                                        d1df1=pds.DataFrame(dic,index=[0])
                                        d2df=d2df.append(d1df1,ignore_index=True)
                                        dic.update(dic2)
                                        d1df1=pds.DataFrame(dic,index=[0])
                                        d2df=d2df.append(d1df1,ignore_index=True)

                                    elif value==3:
                                        partial_keyword="ASSESSED    COPY"
                                        index_found=next((index for index, item in enumerate(text1) if partial_keyword in item),None)
                                        PN1=str(text1[index_found])
                                        PN1=PN1.split(' ')
                                        PN1=PN1[6]
                                        PN1=PN1[4:]

                                        PN2=text1[index_found+1]
                                        PN2=PN2.split(' ')
                                        PN2=PN2[0]

                                        PN3=text1[index_found+2]
                                        PN3=PN3.split(' ')
                                        PN3=PN3[0]

                                        partial_keyword="NOEXCISE"
                                        index_found=next((index for index, item in enumerate(text1) if partial_keyword in item),None)
                                        QTY1=str(text1[index_found+2])
                                        QTY1=QTY1.split(' ')
                                        QTY1=QTY1[0][8:]

                                        partial_keyword="NOEXCISE"
                                        index_found=next((index for index, item in enumerate(text1) if partial_keyword in item),None)
                                        QTY2=str(text1[index_found+3])

                                        partial_keyword="NOEXCISE"
                                        index_found=next((index for index, item in enumerate(text1) if partial_keyword in item),None)
                                        QTY3=str(text1[index_found+4])
                                        QTY3=QTY3.split(' ')
                                        QTY3=QTY3[0][:-3]

                                        dic1={'PN':PN1,'QTY':QTY1}
                                        dic2={'PN':PN2,'QTY':QTY2}
                                        dic3={'PN':PN3,'QTY':QTY3}
                                        dic.update(dic1)
                                        d1df1=pds.DataFrame(dic,index=[0])
                                        d2df=d2df.append(d1df1,ignore_index=True)
                                        dic.update(dic2)
                                        d1df1=pds.DataFrame(dic,index=[0])
                                        d2df=d2df.append(d1df1,ignore_index=True)
                                        dic.update(dic3)
                                        d1df1=pds.DataFrame(dic,index=[0])
                                        d2df=d2df.append(d1df1,ignore_index=True)

                        elif result2==None and result_indices==None:
                            dpath=r'Exception/'
                            dest_path=f'{dpath}{filename}'
                            shutil.move(path,dest_path)

                        dpath=r'Destination_Path/'
                        dest_path=f'{dpath}{filename}'
                        shutil.move(path,dest_path)
            else:
                print("Source is empty")
except Exception as e:
    print("Exception",e)
print("BOE Dataframe")
print(d2df)
e_path=r'BOE-Raw_Data.xlsx'
existing_file=pds.read_excel(e_path)
book=load_workbook(e_path)
writer=pds.ExcelWriter(e_path,engine='openpyxl')
writer.book=book

writer.sheets=dict((ws.title,ws) for ws in book.worksheets)
d2df.to_excel(writer,sheet_name='BOE',index=False,header=False,startrow=writer.sheets['BOE'].max_row)
writer.save()
print("Process Done")
